## Preparação dos Dados para realizar o Ajuste Fino


In [2]:
#importando as funções importantes para trabalhar com os dados
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import json

##### Preparando os dados

Quando vamos realizar um Fine-Tune, antes de qualquer coisa, precisamos preparar os dados que serão utilizados para realizar o ajuste do modelo.

Os dados utilizados neste algoritmo foram criados com base nos beneficios oferecidos pela Secretaria de Estado de Desenvolvimento Social de Goiás. Essas informações podem ser consultadas neste [link]("https://goias.gov.br/social/#").

O arquivo na extensão .json que foi utilizado está disponivel para download neste [link]().

Extrai as informações do arquivo e atribui á um dicionario

Etiquetei cada prompt com o nome do programa correspondente para melhorar a identificação do programa que se refere cada pergunta

In [ ]:
dataset_por_programa = {} #criando um dicionario para receber as informações do arquivo json "relacionados3.json"

# Carregar o arquivo JSON
with open('relacionados3.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Percorrer cada programa no arquivo JSON
for programa, lista_perguntas_respostas in data.items():
    # Criar uma entrada para o programa no dicionário
    dataset_por_programa[programa] = []
    # Percorrer cada item (dicionário) etiquetar prompts e completions
    for item in lista_perguntas_respostas:
        dataset_por_programa[programa].append({'prompt': f"{programa} : {item['prompt']}", 'completion': item['completion']})


In [ ]:
print(dataset_por_programa)

Neste código, estamos iterando sobre o dicionário e extraindo as perguntas e respostas para cada programa. Em seguida, estruturamos essas perguntas e respostas em pares e os adicionamos à lista perguntas_respostas.

In [ ]:
perguntas_respostas = []

for programa, lista_perguntas in dataset_por_programa.items():
    for pergunta_resposta in lista_perguntas:
        pergunta = pergunta_resposta["prompt"]
        respostas = pergunta_resposta["completion"]
        for resposta in respostas:
            # Adiciona o par pergunta-resposta à lista
            perguntas_respostas.append((pergunta, resposta))

print(perguntas_respostas[:5])  # Exibe os primeiros 5 pares pergunta-resposta

### Testando fazer com csv

In [ ]:
!pip install pandas
import pandas as pd

In [ ]:
# Carregar o arquivo CSV
df = pd.read_csv('dataset.csv')

# Dividir o conjunto de dados em conjuntos de treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Salvar os conjuntos de treinamento e teste em novos arquivos CSV
train_df.to_csv('treino.csv', index=False)
test_df.to_csv('teste.csv', index=False)